# 🔐 Secrets Project – Hashiranje in šifriranje

Ta dokument prikazuje uporabo **bcrypt** za hashiranje gesel in uporabo sejnih piškotkov ter OAuth za avtentikacijo.
Primeri temeljijo na Node.js / Express projektu.

---


## Kako zagnati projekt (lokalno)

```bash
npm install
npm run start   # ali: node server.js
# pred zagonom ustvari .env iz .env.example in nastavi vrednosti (SESSION_SECRET, PG_*, GOOGLE_*)
```

Če uporabljaš PostgreSQL, poskrbi, da imaš konfigurirane okoljske spremenljivke (`PG_USER`, `PG_HOST`, `PG_DATABASE`, `PG_PASSWORD`, `PG_PORT`).


## 📦 Glavne odvisnosti

- **express**, **passport**, **passport-local**, **express-session**
- **bcrypt** (hashiranje gesel)
- **pg** (PostgreSQL)
- **passport-google-oauth** ali **passport-google-oauth20** (Google prijava)
- **dotenv** (nalaganje `.env`)


## Prikaz kode (server.js)

> Spodaj je prikaz izseka strežniške kode. Koda je dodana kot *fenced code block* (```javascript) zato se v tem zvezku ne izvaja
> in je namenjena **branju**. Samo *server.js* dejansko zaganja Express strežnik.

```javascript
import express from "express";
import bodyParser from "body-parser";
import pg from "pg";
import bcrypt from "bcrypt";
import passport from "passport";
import { Strategy } from "passport-local";
import GoogleStrategy from "passport-google-oauth2";
import session from "express-session";
import env from "dotenv";

const app = express();
const port = 3000;
const saltRounds = 10;
env.config();

app.use(
  session({
    secret: process.env.SESSION_SECRET,
    resave: false,
    saveUninitialized: true,
  })
);
app.use(bodyParser.urlencoded({ extended: true }));
app.use(express.static("public"));

app.use(passport.initialize());
app.use(passport.session());

const db = new pg.Client({
  user: process.env.PG_USER,
  host: process.env.PG_HOST,
  database: process.env.PG_DATABASE,
  password: process.env.PG_PASSWORD,
  port: process.env.PG_PORT,
});
db.connect();

app.get("/", (req, res) => {
  res.render("home.ejs");
});

app.get("/login", (req, res) => {
  res.render("login.ejs");
});

app.get("/register", (req, res) => {
  res.render("register.ejs");
});

app.get("/logout", (req, res) => {
  req.logout(function (err) {
    if (err) {
      return next(err);
    }
    res.redirect("/");
  });
});

app.get("/secrets", async (req, res) => {
  if (req.isAuthenticated()) {
    try {
      const result = await db.query(
        "SELECT secret FROM users WHERE email = $1",
      [req.user.email]);
      const secret = result.rows[0]?.secret;
      if (secret) {
        res.render("secrets.ejs", { secret: secret});
      } else {
        res.render("secret.ejs", {secret: "There’s no worse moment than when you suddenly remember something you said in 2014 and have to survive a mini shame attack while washing the dishes."})
      }
    } catch (error) {
      console.log(error);
    }
  } else {
    res.redirect("/login");
  }
});

app.get("/submit", function (req, res) {
  if(req.isAuthenticated()) {
    res.render("submit.ejs");
  } else {
    res.redirect("/login");
  }
});

app.get(
  "/auth/google",
  passport.authenticate("google", {
    scope: ["profile", "email"],
  })
);

app.get(
  "/auth/google/secrets",
  passport.authenticate("google", {
    successRedirect: "/secrets",
    failureRedirect: "/login",
  })
);

app.post(
  "/login",
  passport.authenticate("local", {
    successRedirect: "/secrets",
    failureRedirect: "/login",
  })
);

app.post("/register", async (req, res) => {
  const email = req.body.username;
  const password = req.body.password;

try {
    const checkResult = await db.query("SELECT * FROM users WHERE email = $1", [
      email,
    ]);

    if (checkResult.rows.length > 0) {
      res.redirect("/login");
    } else {
      bcrypt.hash(password, saltRounds, async (err, hash) => {
        if (err) {
          console.error("Error hashing password:", err);
        } else {
          const result = await db.query(
            "INSERT INTO users (email, password) VALUES ($1, $2) RETURNING *",
            [email, hash]
          );
          const user = result.rows[0];
          req.login(user, (err) => {
            console.log("success");
            res.redirect("/secrets");
          });
        }
      });
    }
  } catch (err) {
    console.log(err);
  }
});

app.post("/submit", async function (req, res) {
  const submittedSecret = req.body.secret;
  try {
    await db.query("UPDATE users SET secret = $1 WHERE email = $2", [
      submittedSecret,
      req.user.email,
    ]);
    res.redirect("/secrets");
  } catch (err) {
    console.log(err);
  }
});

passport.use(
  "local",
  new Strategy(async function verify(username, password, cb) {
    try {
      const result = await db.query("SELECT * FROM users WHERE email = $1 ", [
        username,
      ]);
      if (result.rows.length > 0) {
        const user = result.rows[0];
        const storedHashedPassword = user.password;
        bcrypt.compare(password, storedHashedPassword, (err, valid) => {
          if (err) {
            console.error("Error comparing passwords:", err);
            return cb(err);
          } else {
            if (valid) {
              return cb(null, user);
            } else {
              return cb(null, false);
            }
          }
        });
      } else {
        return cb("User not found");
      }
    } catch (err) {
      console.log(err);
    }
  })
);

passport.use(
  "google",
  new GoogleStrategy(
    {
      clientID: process.env.GOOGLE_CLIENT_ID,
      clientSecret: process.env.GOOGLE_CLIENT_SECRET,
      callbackURL: "http://localhost:3000/auth/google/secrets",
      userProfileURL: "https://www.googleapis.com/oauth2/v3/userinfo",
    },
    async (accessToken, refreshToken, profile, cb) => {
      try {
        const result = await db.query("SELECT * FROM users WHERE email = $1", [
          profile.email,
        ]);
        if (result.rows.length === 0) {
          const newUser = await db.query(
            "INSERT INTO users (email, password) VALUES ($1, $2)",
            [profile.email, "google"]
          );
          return cb(null, newUser.rows[0]);
        } else {
          return cb(null, result.rows[0]);
        }
      } catch (err) {
        return cb(err);
      }
    }
  )
);
passport.serializeUser((user, cb) => {
  cb(null, user);
});

passport.deserializeUser((user, cb) => {
  cb(null, user);
});

app.listen(port, () => {
  console.log(`Server running on port ${port}`);
});
```


## Opombe

- Gesla **vedno hashiraj** (npr. `bcrypt`), **nikoli** ne šifriraj gesel.
- Sejne piškotke nastavi `httpOnly`, `sameSite` in na produkciji `secure: true`.
- Za Google OAuth raje uporabi paket **`passport-google-oauth20`** in uvoz `import { Strategy as GoogleStrategy } from "passport-google-oauth20";`.
- Če uporabljaš `EJS`, dodaj `app.set("view engine", "ejs")` in `app.set("views", "./views")`.
- Okoljske spremenljivke imej usklajene (`PGUSER` vs. `PG_USER` ipd.).
